# Create gene vectors from CNA data

Use cooccurrence statistics in the discrete CNA data to create gene vectors. 


https://www.kaggle.com/code/kenshoresearch/kdwd-pmi-word-vectors

https://aclanthology.org/Q15-1016/ (LGD15)



In [ ]:
from collections import Counter
import itertools
import json
import math
from pathlib import Path
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
from scipy import sparse
from scipy.sparse import linalg
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

In [ ]:
from hack4nf.synapse import get_dataset
from hack4nf.genie_utils import (
    read_clinical_patient, 
    read_clinical_sample, 
    read_mutations_extended,
    read_cna,
    read_cna_seg,
    SYNIDS,
    dme_to_cravat,
    get_cna_norms,
    get_melted_cna,
)

In [ ]:
#from IPython.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
pd.set_option('display.max_columns', 100)

In [ ]:
genie_dataset_version = "genie_12.0"
#genie_dataset_version = "genie_13.3"

In [ ]:
syn_file_paths = {
    'data_clinical_patient': get_dataset(SYNIDS[genie_dataset_version]['data_clinical_patient']).path,
    'data_clinical_sample': get_dataset(SYNIDS[genie_dataset_version]['data_clinical_sample']).path,
    'data_mutations_extended': get_dataset(SYNIDS[genie_dataset_version]['data_mutations_extended']).path,
    'data_CNA': get_dataset(SYNIDS[genie_dataset_version]['data_CNA']).path,
    'data_cna_hg19_seg': get_dataset(SYNIDS[genie_dataset_version]['data_cna_hg19_seg']).path,
}
syn_file_paths

# GENIE - Data CNA (Discrete Copy Number Alteration Data)

https://docs.cbioportal.org/file-formats/#discrete-copy-number-data

For each gene-sample combination, a copy number level is specified:

* "-2" is a deep loss, possibly a homozygous deletion
* "-1" is a single-copy loss (heterozygous deletion)
* "0" is diploid
* "1" indicates a low-level gain
* "2" is a high-level amplification.

In [ ]:
df_cna = read_cna(syn_file_paths['data_CNA'])

In [ ]:
df_cna

# Create CNA "Sentences" 

We are going to create Gene vectors by treating them like words in sentences. 

# Pointwise Mutual Information Matrices


## Notation


We assume a collection of words $w \in V_W$ and their
contexts $c \in V_C$, where $V_W$ and $V_C$
are the word and context vocabularies, and denote
the collection of observed word-context pairs as $D$.

We use $\#(w,c)$ to denote the number of times the pair
$(w,c)$ appears in $D$. Similarly, 

$$
\begin{align}
\#(w) = \sum_{c^{\prime} \in V_C} \#(w, c^{\prime})
\\
\#(c) = \sum_{w^{\prime} \in V_W} \#(w^{\prime}, c)
\end{align}
$$

are the number of times $w$ and $c$ occurred in $D$, respectively.

## Contexts

$D$ is commonly obtained by taking a
corpus $w_1$, $w_2$, . . . , $w_n$ and defining the contexts
of word $w_i$ as the words surrounding it in an 
$L$-sized window $w_{i−L}$, . . . , $w_{i−1}$, $w_{i+1}$, . . . , $w_{i+L}$.

In our case, the corpus will be genes and their contexts will be 
other genes that they co-occurr with. 


## Definitions

$$
\begin{align}
PMI(w, c) = 
\log \frac
{\hat{P}(w,c)}
{\hat{P}(w)\hat{P}(c)} =
\log \frac
{\#(w,c) \, \cdot \lvert D \rvert}
{\#(w) \cdot \#(c)}
\\
\\
PPMI(w, c) = {\rm max} \left[ PMI(w, c), 0 \right]
\\
\\
\#(w) = \sum_{c^{\prime}} \#(w, c^{\prime}),
\quad
\#(c) = \sum_{w^{\prime}} \#(w^{\prime}, c)
\end{align}
$$


## Context Distribution Smoothing

$$
\begin{align}
PMI_{\alpha}(w, c) = 
\log \frac
{\hat{P}(w,c)}
{\hat{P}(w)\hat{P}_{\alpha}(c)}
\\
\\
\hat{P}_{\alpha}(c) = 
\frac
{\#(c)^{\alpha}}
{\sum_c \#(c)^{\alpha}}
\end{align}
$$

In [ ]:
df_cna_melted = get_melted_cna(df_cna, drop_nan=True, drop_zero=True)

In [ ]:
df_cna_melted

In [ ]:
ser_cna_tokens = df_cna_melted.groupby('SAMPLE_ID').apply(lambda x: list(zip(x['hugo'], x['dcna'])))

In [ ]:
ser_cna_tokens

# Create Gene Cooccurence and PMI Matrices

In [ ]:
indx_to_gene = {ii: gene for ii, gene in enumerate(df_cna.columns)}
gene_to_indx = {gene: ii for ii, gene in indx_to_gene.items()}

In [ ]:
unigram_counts = Counter()
for row in tqdm(ser_cna_tokens):
    for gene, weight in row:
        unigram_counts[gene] += abs(weight)

### Zipfian Plot 

In [ ]:
ranks = list(range(1, len(unigram_counts) + 1))
weights = [unigram[1] for unigram in unigram_counts.most_common()]

In [ ]:
_ = plt.plot(ranks, np.log10(weights)) 

In [ ]:
# Here we create (word-gene, context-gene) pairs by forming 
# every possile permuation of size 2 for all the genes that 
# have non-zero count number variation in the same sample. 
# for weight we use the norm of the vector created by the 
# discrete CNA values for the two genes.  
skipgram_weights = Counter()
for row in tqdm(ser_cna_tokens):
    for combo in itertools.permutations(row, 2):
        (gene_left, weight_left), (gene_right, weight_right) = combo
        weight = math.sqrt(weight_left**2 + weight_right**2)
        skipgram = (gene_left, gene_right)
        skipgram_weights[skipgram] += weight

In [ ]:
skipgram_weights.most_common(10)

### Create Sparse Co-Occurrence Matrix

In [ ]:
row_indxs = []
col_indxs = []
dat_values = []
for (gene1, gene2), sg_weight in tqdm(skipgram_weights.items()):    
    row_indxs.append(gene_to_indx[gene1])
    col_indxs.append(gene_to_indx[gene2])
    dat_values.append(sg_weight)
gg_weight_mat = sparse.csr_matrix((dat_values, (row_indxs, col_indxs)))

In [ ]:
gg_weight_mat

In [ ]:
# normalize rows
gg_weight_mat_l2 = normalize(gg_weight_mat, norm='l2', axis=1)

In [ ]:
# demonstrate normalization
irow=10
row = gg_weight_mat_l2.getrow(irow).toarray().flatten()
print(np.sqrt((row*row).sum()))

row = gg_weight_mat.getrow(irow).toarray().flatten()
print(np.sqrt((row*row).sum()))

### Pointwise Mutual Information Matrices

In [ ]:
word = "NF1"
context = "KRAS"
word_indx = gene_to_indx[word]
context_indx = gene_to_indx[context]
print('#(w,c) for ({},{}) from skipgrams: {}'.format(
    word, context, skipgram_weights[(word, context)]))
print('#(w,c) for ({},{}) from count_matrix: {}'.format(
    word, context, gg_weight_mat[word_indx, context_indx]))

In [ ]:
sum_over_words = np.array(gg_weight_mat.sum(axis=0)).flatten()    # sum over rows
sum_over_contexts = np.array(gg_weight_mat.sum(axis=1)).flatten() # sum over columns

pound_w_check1 = gg_weight_mat.getrow(word_indx).sum()
pound_w_check2 = sum_over_contexts[word_indx]
print('#(w) for "{}" from getrow then sum: {}'.format(word, pound_w_check1))
print('#(w) for "{}" from sum_over_contexts: {}'.format(word, pound_w_check2))

pound_c_check1 = gg_weight_mat.getcol(context_indx).sum()
pound_c_check2 = sum_over_words[context_indx]
print('#(c) for "{}" from getcol then sum: {}'.format(context, pound_c_check1))
print('#(c) for "{}" from sum_over_words: {}'.format(context, pound_c_check2))

In [ ]:
def get_ppmi_matrix(skipgram_weights, gg_weight_mat, gene_to_indx, alpha=0.75):

    # for standard PPMI
    DD = gg_weight_mat.sum()
    sum_over_words = np.array(gg_weight_mat.sum(axis=0)).flatten()
    sum_over_contexts = np.array(gg_weight_mat.sum(axis=1)).flatten()
    
        
    # for context distribution smoothing (cds)
    sum_over_words_alpha = sum_over_words**alpha
    Pc_alpha_denom = np.sum(sum_over_words_alpha)
        
    row_indxs = []
    col_indxs = []
    ppmi_dat_values = []   # positive pointwise mutual information
    
    for skipgram in tqdm(
        skipgram_weights.items(), 
        total=len(skipgram_weights), 
        desc='building ppmi matrix row,col,dat'
    ):
        
        (word, context), pound_wc = skipgram
        word_indx = gene_to_indx[word]
        context_indx = gene_to_indx[context]
        
        pound_w = sum_over_contexts[word_indx]
        pound_c = sum_over_words[context_indx]
        pound_c_alpha = sum_over_words_alpha[context_indx]

        Pwc = pound_wc / DD
        Pw = pound_w / DD
        Pc = pound_c / DD
        Pc_alpha = pound_c_alpha / Pc_alpha_denom

        pmi = np.log2(Pwc / (Pw * Pc_alpha))
        ppmi = max(pmi, 0)
        
        row_indxs.append(word_indx)
        col_indxs.append(context_indx)
        ppmi_dat_values.append(ppmi)

    mat = sparse.csr_matrix((ppmi_dat_values, (row_indxs, col_indxs)))
    return mat

In [ ]:
ppmi_mat = get_ppmi_matrix(skipgram_weights, gg_weight_mat, gene_to_indx)

In [ ]:
ppmi_mat

In [ ]:
def gg_sim(gene, mat, topn=10):
    """Calculate topn most similar genes to input gene"""
    indx = gene_to_indx[gene]
    if isinstance(mat, sparse.csr_matrix):
        v1 = mat.getrow(indx)
    else:
        v1 = mat[indx:indx+1, :]
    sims = cosine_similarity(mat, v1).flatten()
    sindxs = np.argsort(-sims)
    sim_scores = [(indx_to_gene[sindx], sims[sindx]) for sindx in sindxs[0:topn]]
    return sim_scores

In [ ]:
gg_sim("NF1", ppmi_mat)

In [ ]:
gg_sim("NF2", ppmi_mat)

In [ ]:
gg_sim("KRAS", ppmi_mat, topn=20)

In [ ]:
gg_sim("BRAF", ppmi_mat, topn=20)

In [ ]:
embedding_size = 200
uu, ss, vv = linalg.svds(ppmi_mat, embedding_size)

In [ ]:
print('vocab size: {}'.format(len(unigram_counts)))
print('ppmi size: {}'.format(ppmi_mat.shape))
print('embedding size: {}'.format(embedding_size))
print('uu.shape: {}'.format(uu.shape))
print('ss.shape: {}'.format(ss.shape))
print('vv.shape: {}'.format(vv.shape))

In [ ]:
p = 1.0
svd_gene_vecs = uu.dot(np.diag(ss**p))
print(svd_gene_vecs.shape)

In [ ]:
nbins = 20
fig, axes = plt.subplots(2, 2, figsize=(16,14), sharey=False)

ax = axes[0,0]
xx = gg_weight_mat.data
ax.hist(xx, bins=nbins, density=True, log=True)
ax.set_xlabel('gene-gene weights')
ax.set_ylabel('fraction')

ax = axes[0,1]
xx = gg_weight_mat_l2.data
ax.hist(xx, bins=nbins, density=True, log=True)
ax.set_xlim(-0.05, 1.05)
ax.set_xlabel('gene-gene weights L2')

ax = axes[1,0]
xx = ppmi_mat.data
ax.hist(xx, bins=nbins, density=True, log=True)
ax.set_xlabel('PPMI')
ax.set_ylabel('fraction')

ax = axes[1,1]
xx = svd_gene_vecs.flatten()
ax.hist(xx, bins=nbins, density=True, log=True)
ax.set_xlabel(f'SVD(p={p})-PPMI')

fig.suptitle('Distribution of Embedding Matrix Values');

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
svd_2d = TSNE(
    n_components=2, 
    init='pca',
    learning_rate='auto',
    random_state=3847,
).fit_transform(svd_gene_vecs)

In [ ]:
word='NF1'
size = 3
indx = gene_to_indx[word]
cen_vec = svd_2d[indx,:]
dxdy = np.abs(svd_2d - cen_vec) 
bmask = (dxdy[:,0] < size) & (dxdy[:,1] < size)
sub = svd_2d[bmask]


fig, ax = plt.subplots(figsize=(15,15))
ax.scatter(sub[:,0], sub[:,1])
ax.set_xlim(cen_vec[0] - size, cen_vec[0] + size)
ax.set_ylim(cen_vec[1] - size, cen_vec[1] + size)
for ii in range(len(indx_to_gene)):
    if not bmask[ii]:
        continue
    plt.annotate(
        indx_to_gene[ii],
        xy=(svd_2d[ii,0], svd_2d[ii,1]),
        xytext=(5, 2),
        textcoords='offset points',
        ha='right',
        va='bottom')

# Vectors for Projector

In [ ]:
svd_gene_vecs

In [ ]:
df_vecs = pd.DataFrame(svd_gene_vecs)
df_vecs.to_csv('pmi_svd_gene_cna_vecs.tsv', sep='\t', index=False, header=False)

# Metadata for Projector

In [ ]:
cna_gene_l2 = get_cna_norms(df_cna, axis=0).sort_values()
cna_samp_l2 = get_cna_norms(df_cna, axis=1).sort_values()

In [ ]:
cna_gene_l2

In [ ]:
df_ras = pd.read_excel('../data/nci-ras-initiative/ras-pathway-gene-names.xlsx')

In [ ]:
df_ras

In [ ]:
df_meta = pd.DataFrame(df_cna.columns).set_index("Hugo_Symbol")

In [ ]:
df_meta['cna_norm'] = cna_gene_l2
df_meta = df_meta.reset_index()

In [ ]:
df_meta

In [ ]:
df_meta = pd.merge(
    df_meta, 
    df_ras[['Gene name']], 
    left_on='Hugo_Symbol', 
    right_on='Gene name', 
    how='left',
).rename(columns={"Gene name": "is_RAS_path"})

In [ ]:
df_meta

In [ ]:
bmask = df_meta['is_RAS_path'].isnull()
df_meta.loc[~bmask, 'is_RAS_path'] = 1
df_meta.loc[bmask, 'is_RAS_path'] = 0

In [ ]:
df_meta[df_meta['is_RAS_path']==1]

In [ ]:
df_ras

In [ ]:
df_meta.to_csv('pmi_svd_gene_cna_meta.tsv', sep='\t', index=False)

In [ ]:
set(df_ras['Gene name']) - set(df_meta['Hugo_Symbol'])

In [ ]:
df_meta['Hugo_Symbol'].isin(['DHFR']).sum()